# McNemar Test

## Import All Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar 
import os

## Read All the Data

In [2]:
dir = '/Users/angelaoryza/Documents/TA/noisy-rnnids/rnnids-py/results/vector/mcnemar/ftp'
subdir = os.listdir(dir)

In [3]:
df_dict = {}
df_dict['bi-lstm'] = {}
df_dict['lstm'] = {}
for file in subdir:
    if file.endswith('.csv'):
        df = pd.read_csv(f'./{file}')
        file = file.split('.csv')[0]
        if "bi-lstm" in file:
            file = file.replace('bi-lstm_', '')
            df_dict['bi-lstm'][file] = df
        else:
            file = file.replace('lstm_', '')
            df_dict['lstm'][file] = df

In [4]:
df_dict['bi-lstm']['pred_0.0']

,Unnamed: 0,id,b_mean,b_iqr,b_zscore,f_zscore,f_mean,f_iqr
0,0,175.45.176.0-45235-149.171.126.16-21-tcp,1,1,1,1,1,1
1,1,175.45.176.3-1025-149.171.126.11-21-tcp,1,1,1,1,1,1
2,2,175.45.176.1-28136-149.171.126.11-21-tcp,1,1,1,1,1,1
3,3,175.45.176.2-43663-149.171.126.15-21-tcp,1,1,1,1,1,1
4,4,175.45.176.3-64709-149.171.126.15-21-tcp,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
25298,25298,59.166.0.8-55795-149.171.126.6-21-tcp,0,0,0,0,0,0
25299,25299,149.171.126.1-21-59.166.0.7-7753-tcp,0,1,1,1,0,1
25300,25300,59.166.0.3-7585-149.171.126.2-21-tcp,0,0,0,0,0,0
25301,25301,59.166.0.1-27560-149.171.126.3-21-tcp,0,1,1,1,0,1


In [5]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        val.drop('Unnamed: 0', axis=1, inplace=True)

## Data Exploration

In [6]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        temp = val[val.duplicated()]['id'].unique()
        print(f'There are {len(temp)} duplicates data in df {k}')

There are 390 duplicates data in df pred_0.0039
There are 386 duplicates data in df pred_0.0005
There are 385 duplicates data in df pred_0.0015
There are 505 duplicates data in df pred_0.0
There are 392 duplicates data in df pred_0.0025
There are 376 duplicates data in df pred_0.003
There are 389 duplicates data in df pred_0.002
There are 390 duplicates data in df pred_0.0035
There are 386 duplicates data in df pred_0.001
There are 388 duplicates data in df pred_0.002
There are 378 duplicates data in df pred_0.003
There are 387 duplicates data in df pred_0.001
There are 493 duplicates data in df pred_0.0
There are 389 duplicates data in df pred_0.0035
There are 389 duplicates data in df pred_0.0025
There are 386 duplicates data in df pred_0.0015
There are 389 duplicates data in df pred_0.0039
There are 386 duplicates data in df pred_0.0005


In [7]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        count = 0
        for t in temp:
            srs = val[val['id']==t][['b_mean', 'b_iqr', 'b_zscore', 'f_mean', 'f_iqr', 'f_zscore' ]].nunique()
            if (srs > 1).any():
                print(t)
                count += 1
                break
            else:
                continue
        if count == 0:
            print(f'All duplicated values in df {key}-{k} have the same prediction')
        else:
            print(f'Not all duplicated values in df {key}-{k} have the same prediction')

175.45.176.1-52899-149.171.126.10-21-tcp
Not all duplicated values in df bi-lstm-pred_0.0039 have the same prediction
175.45.176.0-2939-149.171.126.15-21-tcp
Not all duplicated values in df bi-lstm-pred_0.0005 have the same prediction
175.45.176.2-60931-149.171.126.11-21-tcp
Not all duplicated values in df bi-lstm-pred_0.0015 have the same prediction
149.171.126.0-21-59.166.0.9-40745-tcp
Not all duplicated values in df bi-lstm-pred_0.0 have the same prediction
149.171.126.9-21-59.166.0.2-32919-tcp
Not all duplicated values in df bi-lstm-pred_0.0025 have the same prediction
149.171.126.0-21-59.166.0.9-40745-tcp
Not all duplicated values in df bi-lstm-pred_0.003 have the same prediction
175.45.176.2-60931-149.171.126.11-21-tcp
Not all duplicated values in df bi-lstm-pred_0.002 have the same prediction
175.45.176.1-1238-149.171.126.13-21-tcp
Not all duplicated values in df bi-lstm-pred_0.0035 have the same prediction
175.45.176.2-60931-149.171.126.11-21-tcp
Not all duplicated values in df

## Data Pre-Processing

In [8]:
def preprocess(df):
    df = df.sort_values(by='id')
    df.drop_duplicates(inplace=True)
    return df

In [9]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        val = preprocess(val)

### Merge DF

In [10]:
df_merge = dict()
for key, df in df_dict['lstm'].items():
    x = df_dict['bi-lstm'][key]
    df_all = pd.merge(x, df, on='id', how='inner')
    df_all = df_all.sort_values(by='id')
    df_merge[key] = df_all
        

- Plot F1-Score untuk tiap tiap threshold
- Cari trend regresinya, Hitung gradiennya untuk tiap threshold.
- Dilakukan untuk setiap protokol
- Rekap McNemar

- Vectorizing:


## Run McNemar Test

In [11]:
def mcnemar_test(df, x , y, key):
    a = len(df[(df[x]==0) & (df[y]==0)])
    b = len(df[(df[x]==0) & (df[y]==1)])
    c = len(df[(df[x]==1) & (df[y]==0)])
    d = len(df[(df[x]==1) & (df[y]==1)])

    table = np.array([
    [a,b],
    [c,d]
    ])

    print(key)
    print(table)
    result = mcnemar(table)
    print(result)
    return result.pvalue


### For Binary Score

#### For B-Mean

In [12]:
result = {}
result['b-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_mean_x', 'b_mean_y', key)
    result['b-mean'][key] = test

pred_0.002
[[24430   191]
 [  179  1947]]
pvalue      0.5674740213663277
statistic   179.0
pred_0.003
[[23935   340]
 [  285  2187]]
pvalue      0.03069084889697326
statistic   285.0
pred_0.001
[[24381   195]
 [  166  2005]]
pvalue      0.14045935551164945
statistic   166.0
pred_0.0
[[24077   213]
 [  267  2190]]
pvalue      0.015471317174642852
statistic   213.0
pred_0.0035
[[24602   207]
 [  164  1774]]
pvalue      0.029081665049916328
statistic   164.0
pred_0.0025
[[24330   410]
 [  224  1783]]
pvalue      1.3608462331755516e-13
statistic   224.0
pred_0.0015
[[24364   185]
 [  208  1990]]
pvalue      0.26708220847031827
statistic   185.0
pred_0.0039
[[24642   245]
 [  152  1708]]
pvalue      3.5275971040330817e-06
statistic   152.0
pred_0.0005
[[23622   434]
 [   48  2643]]
pvalue      7.871428050259647e-79
statistic   48.0


#### For B-IQR

In [13]:
result['b-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_iqr_x', 'b_iqr_y', key)
    result['b-iqr'][key] = test

pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0


#### For B-ZScore

In [14]:
result['b-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_zscore_x', 'b_zscore_y', key)
    result['b-zscore'][key] = test

pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0


### For Floating Score

#### For F-Mean

In [15]:
result['f-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_mean_x', 'f_mean_y', key)
    result['f-mean'][key] = test

pred_0.002
[[25031   127]
 [  130  1459]]
pvalue      0.9007479747783873
statistic   127.0
pred_0.003
[[24750   452]
 [  124  1421]]
pvalue      8.893068246103901e-45
statistic   124.0
pred_0.001
[[25080   140]
 [  141  1386]]
pvalue      1.0
statistic   140.0
pred_0.0
[[24582   360]
 [ 1538   267]]
pvalue      6.737011519419362e-173
statistic   360.0
pred_0.0035
[[25103   130]
 [  137  1377]]
pvalue      0.7135487067229689
statistic   130.0
pred_0.0025
[[25018   127]
 [  126  1476]]
pvalue      0.9999999999999998
statistic   126.0
pred_0.0015
[[25052   128]
 [  141  1426]]
pvalue      0.464445493036592
statistic   128.0
pred_0.0039
[[25031   128]
 [  135  1453]]
pvalue      0.7114763589543254
statistic   128.0
pred_0.0005
[[25092   137]
 [  127  1391]]
pvalue      0.57972288694173
statistic   127.0


#### For F-IQR

In [16]:
result['f-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_iqr_x', 'f_iqr_y', key)
    result['f-iqr'][key] = test

pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0


#### For F-ZScore

In [17]:
result['f-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_zscore_x', 'f_zscore_y', key)
    result['f-zscore'][key] = test

pred_0.002
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.003
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.001
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0035
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0025
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0015
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0039
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0
pred_0.0005
[[21298     3]
 [    3  5443]]
pvalue      1.0
statistic   3.0


## Saving Test Result

In [18]:
mcnemar_result = pd.DataFrame.from_dict(result).sort_index(axis=0)
mcnemar_result

,b-mean,b-iqr,b-zscore,f-mean,f-iqr,f-zscore
pred_0.0,1.547132e-02,1.0,1.0,6.737012e-173,1.0,1.0
pred_0.0005,7.871428e-79,1.0,1.0,5.797229e-01,1.0,1.0
pred_0.001,1.404594e-01,1.0,1.0,1.000000e+00,1.0,1.0
pred_0.0015,2.670822e-01,1.0,1.0,4.644455e-01,1.0,1.0
pred_0.002,5.674740e-01,1.0,1.0,9.007480e-01,1.0,1.0
pred_0.0025,1.360846e-13,1.0,1.0,1.000000e+00,1.0,1.0
pred_0.003,3.069085e-02,1.0,1.0,8.893068e-45,1.0,1.0
pred_0.0035,2.908167e-02,1.0,1.0,7.135487e-01,1.0,1.0
pred_0.0039,3.527597e-06,1.0,1.0,7.114764e-01,1.0,1.0


In [19]:
mcnemar_result.to_excel('./ftp-mcnemar.xlsx')